This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [25]:
#import packages 
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [30]:
def make_Dictionary(root_dir):
  print("Creating dictionary from input")
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  print("->extracting each word from emails and adding them to a dictionary")
  #Extract each email from all emails
  for mail in emails:
    with open(mail) as m:
      #extract each line from email
      for line in m:
        #extract each word from email
        words = line.split()
        #add word list of all words
        all_words += words
  #convert to a dictionary
  dictionary = Counter(all_words)
  #create a duplicate as a list
  list_to_remove = list(dictionary)

  print("->removing items from dictionary that include non alphabetical characters or contains more than one word")
  for item in list_to_remove:
    #check if item is non alphabetical
    if item.isalpha() == False:
      del dictionary[item]
    #check if item contains more than one word
    elif len(item) == 1:
      del dictionary[item]
  #return dictionary containing the top 3000 most common words in the emails (most_common function comes from collections package) 
  print("-> returning dictionary containing the 3000 most common words in emails")   
  dictionary = dictionary.most_common(3000)
  return dictionary
            

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [31]:
def extract_features(mail_dir):
  print("extract email features and put in matrix")
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  #Create a matrix(array) filled with zeros that's rows = number of emails and columns = 3000
  features_matrix = np.zeros((len(files),3000))
  #Create a matrix(array) filled with zeros with 1 row and columns = number of emails
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  #iterate over each email
  print("->iterating over each email to create matrices containing the words and word counts")
  for fil in files:
    with open(fil) as fi:
      #extract each line in email - iterate over line in email(line) and track location(i)
      for i, line in enumerate(fi):
        #split word using white space as separater if i = 2
        if i ==2:
          words = line.split()
          #iterate over words
          for word in words:
            #set wordID to 0
            wordID = 0
            #iterate over dictionary and get count each dictionary item - from dictionary function returned dictionary of most common 3000 words in emails and
            for i, d in enumerate(dictionary):
              #set word id to location of word in dictionary if the first item in dictionary is equal to word in line
              if d[0] == word:
                wordID = i
                #add number of times word appears in line to featrues matrix 
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      #split email into strings by /
      filepathTokens = fil.split('/')
      #get the last string
      lastToken = filepathTokens[len(filepathTokens)-1]
      #if the last string starts with spmsg, change the train_label to 1 instead of 0
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        #increase count
        count = count + 1
      #add 1 to doc ID to move down a row in features matrix
      docID = docID + 1
  return features_matrix, train_labels                

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [27]:
#get access to goggle drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
#load data
TRAIN_DIR = '/content/drive/My Drive/2021/Machine Learning/CA 02/Data (2)/train-mails'
TEST_DIR = '/content/drive/My Drive/2021/Machine Learning/CA 02/Data (2)/test-mails'

In [33]:
#call make dictionary function to create dictionary of 3000 most common words
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
#generate words and word frequency matrix
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naive Bayes algorithm .....")
model.fit(features_matrix, labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))

Creating dictionary from input
->extracting each word from emails and adding them to a dictionary
->removing items from dictionary that include non alphabetical characters or contains more than one word
-> returning dictionary containing the 3000 most common words in emails
reading and processing emails from TRAIN and TEST folders
extract email features and put in matrix
->iterating over each email to create matrices containing the words and word counts
extract email features and put in matrix
->iterating over each email to create matrices containing the words and word counts
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


======================= END OF PROGRAM =========================